In [ ]:
!pip install transformers rouge_score
!pip install torch
!pip install pandas
!pip install tensorflow
!pip install contractions
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


method 1: summarize the complete dataset to a new csv and load ...
it will be time consuming

In [ ]:
import os
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
from rouge_score import rouge_scorer

MODEL_NAME = "facebook/bart-large-cnn"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
MAX_INPUT = 1024
MAX_SUMMARY = 128
DATA_PATH = "/content/drive/MyDrive/test_5000.csv"
OUTPUT_PATH = "bart_predictions.csv"

print("Loading model & tokenizer...")
tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()

print("Loading dataset...")
df = pd.read_csv(DATA_PATH, nrows=62)
articles = list(df["article"])
references = list(df["highlights"])

def summarize_batch(texts, max_input=MAX_INPUT, max_summary=MAX_SUMMARY):
    """Summarize a batch of texts using BART."""
    inputs = tokenizer(
        texts,
        max_length=max_input,
        truncation=True,
        return_tensors="pt",
        padding=True
    ).to(DEVICE)

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            num_beams=4,
            length_penalty=2.0,
            max_length=max_summary,
            min_length=30,
            no_repeat_ngram_size=3,
            early_stopping=True
        )

    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

if os.path.exists(OUTPUT_PATH):
    print(f"Loading cached predictions from {OUTPUT_PATH}...")
    results = pd.read_csv(OUTPUT_PATH)
    predictions = list(results["prediction"])
else:
    predictions = []
    for i in range(0, len(articles), BATCH_SIZE):
        batch = articles[i:i + BATCH_SIZE]
        summaries = summarize_batch(batch)
        predictions.extend(summaries)
        print(f"Processed {i + len(batch)}/{len(articles)}")

    #Save predictions so we don’t have to recompute next time
    #if you wat to recompute just delete bart_predictions.csv
    pd.DataFrame({"article": articles, "reference": references, "prediction": predictions}).to_csv(OUTPUT_PATH, index=False)
    print(f"Predictions saved to {OUTPUT_PATH}")

scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
precision, recall, f1 = [], [], []
for hyp, ref in zip(predictions, references):
    scores = scorer.score(ref, hyp)
    s = scores['rouge1']
    precision.append(s.precision)
    recall.append(s.recall)
    f1.append(s.fmeasure)

print("\n=== ROUGE-1 METRICS ===")
print("Precision:", sum(precision) / len(precision))
print("Recall:", sum(recall) / len(recall))
print("F1:", sum(f1) / len(f1))

for idx in range(min(11, len(predictions))):
    print("\nArticle:", articles[idx][:400], "...")
    print("Reference:", references[idx])
    print("Predicted:", predictions[idx])

Loading model & tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Loading dataset...
Processed 4/62
Processed 8/62
Processed 12/62
Processed 16/62
Processed 20/62
Processed 24/62
Processed 28/62
Processed 32/62
Processed 36/62
Processed 40/62
Processed 44/62
Processed 48/62
Processed 52/62
Processed 56/62
Processed 60/62
Processed 62/62
Predictions saved to bart_predictions.csv

=== ROUGE-1 METRICS ===
Precision: 0.39825458248074036
Recall: 0.498801359012385
F1: 0.43268148635457165

Article: Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on  ...
Reference: Experts question if  packed out planes are putting passengers at risk .
U.S consumer advisory group says minimum space must be stipulated .
Safety tests 

### method 2: summarize the indexes we need to display and cache it in a new csv file(cache summaries.csv). fast and efficient

In [ ]:
import os
import time
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

MODEL_NAME = "facebook/bart-large-cnn"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_INPUT = 1024
MAX_SUMMARY = 128
DATA_PATH = "/content/drive/MyDrive/test_5000.csv"
CACHE_FILE = "cached_summaries.csv"
TARGET_INDEXES = [100, 268, 441, 21, 87, 86]#add the indexes which needs to be summarised

print("Initializing model...")
start_time = time.time()

print("Loading tokenizer...")
tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)
print("Tokenizer loaded.")

print("Loading model (this may take a while)...")
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
print(f"Model loaded in {time.time() - start_time:.2f} seconds.")

model.eval()
print(f"Using device: {DEVICE.upper()}")

def load_specific_rows(csv_path, row_indexes):
    row_indexes = sorted(row_indexes)
    skip = lambda x: x != 0 and (x-1) not in row_indexes
    return pd.read_csv(csv_path, skiprows=skip)

def summarize_texts(texts):
    print(f"Summarizing {len(texts)} article(s)...")
    inputs = tokenizer(
        texts,
        max_length=MAX_INPUT,
        truncation=True,
        return_tensors="pt",
        padding=True
    ).to(DEVICE)
    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            num_beams=4,
            length_penalty=2.0,
            max_length=MAX_SUMMARY,
            min_length=30,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
    print("Summarization complete.")
    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

if os.path.exists(CACHE_FILE):
    print(f"Loading cached summaries from {CACHE_FILE}...")
    cached = pd.read_csv(CACHE_FILE)
else:
    cached = pd.DataFrame(columns=["index", "article", "reference", "summary"])

cached_indexes = set(cached["index"]) if not cached.empty else set()
missing_indexes = [i for i in TARGET_INDEXES if i not in cached_indexes]
new_rows = []

if missing_indexes:
    print(f"Fetching {len(missing_indexes)} missing article(s) from CSV...")
    df = load_specific_rows(DATA_PATH, missing_indexes)
    articles = list(df["article"])
    references = list(df["highlights"])
    summaries = summarize_texts(articles)

    for idx, art, ref, summ in zip(missing_indexes, articles, references, summaries):
        new_rows.append({"index": idx, "article": art, "reference": ref, "summary": summ})

    cached = pd.concat([cached, pd.DataFrame(new_rows)], ignore_index=True)
    cached.to_csv(CACHE_FILE, index=False)
    print(f"Cached updated with {len(new_rows)} new summaries.")

for idx in TARGET_INDEXES:
    row = cached[cached["index"] == idx].iloc[0]
    print(f"\nIndex {idx}")
    print("Article (truncated):", row['article'][:500], "...")
    print("Reference:", row['reference'])
    print("Predicted:", row['summary'])


Initializing model...
Loading tokenizer...
Tokenizer loaded.
Loading model (this may take a while)...
Model loaded in 39.48 seconds.
Using device: CUDA
Fetching 6 missing article(s) from CSV...
Summarizing 6 article(s)...
Summarization complete.
Cached updated with 6 new summaries.

Index 100
Article (truncated): Wanted: A police e-fit image of millionaire people trafficker Ermias Ghermay, who is thought to have made £72m with his accomplice Mered Medhanie from smuggling migrants in the last two years . Two millionaire people traffickers have been heard mocking the migrant boat disasters that have resulted in thousands of deaths in the Mediterranean, it has been reported. One of them, an Eritrean called Mered Medhanie, also known as The General, was heard laughing on a police wiretap about overloading mi ...
Reference: Mered Medhanie and Ermias Ghermay 'have made £72m in last two years'
Medhanie heard on police wiretap mocking the fatal overcrowding of ships .
Ghermay reportedly said: 

In [ ]:
def summarize_text(text, max_input_len=512, max_target_len=512, num_beams=4):
    # Tokenize the input text
    inputs = tokenizer(
        text,
        max_length=max_input_len,
        truncation=True,
        return_tensors="pt"
    ).to(DEVICE)

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_target_len,
        num_beams=num_beams,
        early_stopping=True
    )

    # Decode the output
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
random_text = input("enter the text to be summarized:  ")




enter the text to be summarized:  A book is a structured presentation of recorded information, primarily verbal and graphical, through a medium. Originally physical, electronic books and audiobooks are now existent. Physical books are objects that contain printed material, mostly of writing and images. Modern books are typically composed of many pages bound together and protected by a cover, what is known as the codex format; older formats include the scroll and the tablet. As a conceptual object, a book often refers to a written work of substantial length by one or more authors, which may also be distributed digitally as an electronic book (ebook). These kinds of works can be broadly classified into fiction (containing invented content, often narratives) and non-fiction (containing content intended as factual truth). But a physical book may not contain a written work: for example, it may contain only drawings, engravings, photographs, sheet music, puzzles, or removable content like pa

In [ ]:
summary = summarize_text(random_text)

print("Original Text:\n", random_text)
print("\nGenerated Summary:\n", summary)

Original Text:
 A book is a structured presentation of recorded information, primarily verbal and graphical, through a medium. Originally physical, electronic books and audiobooks are now existent. Physical books are objects that contain printed material, mostly of writing and images. Modern books are typically composed of many pages bound together and protected by a cover, what is known as the codex format; older formats include the scroll and the tablet. As a conceptual object, a book often refers to a written work of substantial length by one or more authors, which may also be distributed digitally as an electronic book (ebook). These kinds of works can be broadly classified into fiction (containing invented content, often narratives) and non-fiction (containing content intended as factual truth). But a physical book may not contain a written work: for example, it may contain only drawings, engravings, photographs, sheet music, puzzles, or removable content like paper dolls. The mod